# Part 3: Practical Data Preparation

**Objective:** Handle categorical features using One-Hot Encoding and address class imbalance using SMOTE.

## 1. Setup

Import necessary libraries.

In [163]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
import json


## 2. Data Loading

Load the dataset.


In [164]:
def load_data(file_path):
    """
    Load the synthetic health data from a CSV file.
    
    Args:
        file_path: Path to the CSV file
        
    Returns:
        DataFrame containing the data
    """
    return pd.read_csv(file_path)



## 3. Categorical Feature Encoding

Implement `encode_categorical_features` using `OneHotEncoder`.


In [165]:
def encode_categorical_features(df, column_to_encode='smoker_status'):
    """
    Encode a categorical column using OneHotEncoder.
    
    Args:
        df: Input DataFrame
        column_to_encode: Name of the categorical column to encode
        
    Returns:
        DataFrame with the categorical column replaced by one-hot encoded columns
    """
    # YOUR CODE HERE
    # 1. Extract the categorical column
    col = df[[column_to_encode]]
    # 2. Apply OneHotEncoder
    coded = OneHotEncoder().fit_transform(col)
    # 3. Create new column names
    for i in range(coded.shape[-1]):
        df[column_to_encode + '_' + str(i)] = coded.toarray()[:, i]
    # 4. Replace the original categorical column with the encoded columns
    df = df.drop([column_to_encode], axis=1)

    return df


## 4. Data Preparation

Implement `prepare_data_part3` to handle the train/test split correctly.


In [166]:
def prepare_data_part3(df, test_size=0.2, random_state=42):
    """
    Prepare data with categorical encoding.
    
    Args:
        df: Input DataFrame
        test_size: Proportion of data for testing
        random_state: Random seed for reproducibility
        
    Returns:
        X_train, X_test, y_train, y_test
    """
    # YOUR CODE HERE
    # 1. Encode categorical features using the encode_categorical_features function
    edf = encode_categorical_features(df)
    # 2. Select relevant features (including the one-hot encoded ones) and the target
    X = edf[['age', 'systolic_bp', 'diastolic_bp', 'glucose_level', 'bmi', 'heart_rate', 'smoker_status_0', 'smoker_status_1', 'smoker_status_2']]
    y = edf[['disease_outcome']]
    # 3. Split data into training and testing sets
    X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    # 4. Handle missing values
    X_train = SimpleImputer(missing_values=np.nan, strategy='mean').fit_transform(X_train)
    X_test = SimpleImputer(missing_values=np.nan, strategy='mean').fit_transform(X_test)
    Y_train = SimpleImputer(missing_values=np.nan, strategy='mean').fit_transform(Y_train)
    Y_test = SimpleImputer(missing_values=np.nan, strategy='mean').fit_transform(Y_test)
    
    # Placeholder return - replace with your implementation
    return X_train, X_test, Y_train, Y_test



## 5. Handling Imbalanced Data

Implement `apply_smote` to oversample the minority class.


In [167]:

def apply_smote(X_train, y_train, random_state=42):
    """
    Apply SMOTE to oversample the minority class.
    
    Args:
        X_train: Training features
        y_train: Training target
        random_state: Random seed for reproducibility
        
    Returns:
        Resampled X_train and y_train with balanced classes
    """
    return SMOTE(random_state=random_state).fit_resample(X_train, y_train)



## 6. Model Training and Evaluation

Train a model on the SMOTE-resampled data and evaluate it.


In [168]:

def train_logistic_regression(X_train, y_train):
    """
    Train a logistic regression model.
    
    Args:
        X_train: Training features
        y_train: Training target
        
    Returns:
        Trained logistic regression model
    """
    return LogisticRegression().fit(X_train, y_train)

def calculate_evaluation_metrics(model, X_test, y_test):
    """
    Calculate classification evaluation metrics.
    
    Args:
        model: Trained model
        X_test: Test features
        y_test: Test target
        
    Returns:
        Dictionary containing accuracy, precision, recall, f1, auc, and confusion_matrix
    """
    preds = model.predict(X_test)
    
    return {
        "accuracy": accuracy_score(y_test, preds),
        "precision": precision_score(y_test, preds),
        "recall": recall_score(y_test, preds),
        "f1": f1_score(y_test, preds),
        "auc": roc_auc_score(y_test, preds),
        "confusion_matrix": confusion_matrix(y_test, preds),
    }


## 7. Save Results

Save the evaluation metrics to a text file.


In [169]:
def save_result(result):
    if not os.path.exists('results'):
        os.mkdir('results')
    with open('results/results_part3.txt', 'w') as f:
        json.dump(result, f)


## 8. Main Execution

Run the complete workflow.


In [170]:
# Main execution
if __name__ == "__main__":
    # 1. Load data
    data_file = 'data/synthetic_health_data.csv'
    df = load_data(data_file)
    
    # 2. Prepare data with categorical encoding
    X_train, X_test, y_train, y_test = prepare_data_part3(df)
    
    # 3. Apply SMOTE to balance the training data
    X_train_resampled, y_train_resampled = apply_smote(X_train, y_train)
    
    # 4. Train model on resampled data
    model = train_logistic_regression(X_train_resampled, y_train_resampled)
    
    # 5. Evaluate on original test set
    metrics = calculate_evaluation_metrics(model, X_test, y_test)
    
    # 6. Print metrics
    for metric, value in metrics.items():
        if metric != 'confusion_matrix':
            print(f"{metric}: {value:.4f}")
    del metrics['confusion_matrix']
    
    # 7. Save results
    save_result(metrics)
    
    # 8. Load Part 1 results for comparison
    import json
    try:
        with open('results/results_part1.txt', 'r') as f:
            part1_metrics = json.load(f)
        
        # 9. Compare models
        comparison = compare_models(part1_metrics, metrics)
        print("\nModel Comparison (improvement percentages):")
        for metric, improvement in comparison.items():
            print(f"{metric}: {improvement:.2f}%")
    except FileNotFoundError:
        print("Part 1 results not found. Run part1_introduction.ipynb first.")


accuracy: 0.7694
precision: 0.2738
recall: 0.8252
f1: 0.4111
auc: 0.7943

Model Comparison (improvement percentages):
accuracy: -0.16%
precision: -0.59%
recall: 1.74%
f1: -0.01%
auc: 0.24%


/home/codespace/.local/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



## 9. Compare Results

Implement a function to compare model performance between balanced and imbalanced data.



In [171]:
def compare_models(part1_metrics, part3_metrics):
    """
    Calculate percentage improvement between models trained on imbalanced vs. balanced data.
    
    Args:
        part1_metrics: Dictionary containing evaluation metrics from Part 1 (imbalanced)
        part3_metrics: Dictionary containing evaluation metrics from Part 3 (balanced)
        
    Returns:
        Dictionary with metric names as keys and improvement percentages as values
    """
    # YOUR CODE HERE
    # 1. Calculate percentage improvement for each metric
    # 2. Handle metrics where higher is better (most metrics) and where lower is better
    # 3. Return a dictionary with metric names and improvement percentages
    
    # Placeholder return - replace with your implementation
    return {
        k: 100 * (part3_metrics[k]-part1_metrics[k]) / part1_metrics[k]
        for k in part3_metrics
    }